In [2]:
from tasks import run_md5
from tasks import run_fastqc
from invoke import run
import time
from datetime import datetime
from core.utils import Tibanna
from core import ff_utils

wf_md5 = "md5"
wf_fastqc = "fastqc-0-11-4-1/1"

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
run_md_qc = raw_input("Do you wanna run md5 and/or fastqc if missing? (md5/qc/all/none)")

# status for completion
# there are two flavors of complete signals, before it was output_file_transfer_finished, not it is complete.
# old completed wf runs have former one.
status_done = ['complete']

ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

################
##ADD TO WORKFLOW
# wfr_time = datetime.strptime(wfr_data['date_created'],'%Y-%m-%dT%H:%M:%S.%f+00:00')
# run_hours = int((datetime.now()-wfr_time).total_seconds()/3600)
################

def summarize_file(file_resp):
    qc = False
    file_id = file_resp['accession']
    sequencer = file_resp.get('instrument')
    relations = file_resp.get('related_files')
    status = file_resp.get('status')
    workflows = file_resp.get('workflow_run_inputs')
    first_alias = file_resp.get('aliases',[None])[0]
    pair_no = file_resp.get('paired_end')


    # is there a qc?
    if file_resp.get('quality_metric'):
        qc = True
    # Check workflows for qc fastqc workflow partA
    md5_status = 'did_not_run'
    fastqc_status = 'did_not_run'
    # Assumes workflow_runs come in time ordered list, and grabs the last ones for each wf run
    if workflows:
        for wfr_resp in workflows:
            wfr_resp = ff_utils.get_metadata(wfr_resp['uuid'], connection=ff)
            wfr_name = wfr_resp['display_title']
            if wfr_name.startswith(wf_md5):
                md5_status = wfr_resp.get('run_status')     
            elif wfr_name.startswith(wf_fastqc):
                fastqc_status = wfr_resp.get('run_status')     
                
    # Check for md5 and fastqc, and if not complete, run or report it. 
    # if exclude miseq is on, do this only if sequencer is not miseq

        
    if status in ["uploading"] or md5_status not in status_done:
        print status
        print md5_status
        print status_done
        # if not, shall we run it?
        if run_md_qc in ['md5', 'all']:
            print 'md5 running for', file_resp['accession']
            code_md5= "invoke run_md5 " + env + " " + file_resp['accession'] + " " + file_resp['uuid']
            run(code_md5)
            print ''
            time.sleep(3)
        # user does not want it to be run, so just report
        else:
            print 'md5 run missing for', file_resp['accession']
    # check fastqc if md5 is fine
    else:
        if not qc or fastqc_status not in status_done:
            print qc
            print fastqc_status
            print status_done
            # if not, shall we run it?
            if run_md_qc in ['qc', 'all']:
                print 'fastqc running for', file_resp['accession']
                code_qc= "invoke run_fastqc " + env + " " + file_resp['accession'] + " " + file_resp['uuid']
                run(code_qc)
                print ''    
                time.sleep(3)
            # user does not want it to be run, so just report
            else:
                print 'fastqc run missing for', file_resp['accession'], fastqc_status
                print 
                
                
                
    return

all_files = ff_utils.get_metadata('files-fastq', connection=ff)['@graph']

files_no_qc = [i for i in all_files if not i.get('quality_metric', None) and i['status'] != 'deleted']
print len(all_files)
print len(files_no_qc)
liste = []
counter = 0
for fastq_file in all_files:
    counter += 1
    if fastq_file['status'] == 'deleted':
        continue
    print counter
    file_resp = ff_utils.get_metadata(fastq_file['uuid'], connection=ff, frame='embedded')  
    #Some checks before running
    #check if status is deleted
    if file_resp['status'] == 'deleted':
        continue
    # check if file is in s3
    head_info = tibanna.s3.does_key_exist(file_resp['upload_key'], tibanna.s3.raw_file_bucket)
    if not head_info:
        print file_resp['accession'], "does not have a file in S3"
        continue
    file_info = summarize_file(file_resp)



Do you wanna run md5 and/or fastqc if missing? (md5/qc/all/none)
2668
5
1
True
started
['complete']
fastqc run missing for 4DNFI96XT4FW started

2
True
did_not_run
['complete']
fastqc run missing for 4DNFIWYFMAXR did_not_run

3
True
did_not_run
['complete']
fastqc run missing for 4DNFIXIZEDVB did_not_run

4
True
did_not_run
['complete']
fastqc run missing for 4DNFILOQ8JYX did_not_run

5
True
did_not_run
['complete']
fastqc run missing for 4DNFIG9PFXT7 did_not_run

6
True
did_not_run
['complete']
fastqc run missing for 4DNFIERUVHB7 did_not_run

7
True
did_not_run
['complete']
fastqc run missing for 4DNFIIJ3F86A did_not_run

8
True
did_not_run
['complete']
fastqc run missing for 4DNFI5PBE2HV did_not_run

9
True
did_not_run
['complete']
fastqc run missing for 4DNFI7FOUE8B did_not_run

10
True
did_not_run
['complete']
fastqc run missing for 4DNFI6HRNY45 did_not_run

11
True
did_not_run
['complete']
fastqc run missing for 4DNFID5XUAWS did_not_run

12
True
did_not_run
['complete']
fastqc run

KeyboardInterrupt: 